## **Dependencies**

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

## **Splitting Cam Images (928 x 1280) into 2 Total Images: 1 Multispectral Image (928 x 640) and 1 RGB Image (928 x 640)**

In [ ]:
def process_rgb_images(input_folder, output_folder_rgb):
    # List all camera folders
    cam_folders = [f'cam{i}' for i in range(1, 9)]

    for cam in cam_folders:
        cam_path = os.path.join(input_folder, cam)
        output_cam_rgb_path = os.path.join(output_folder_rgb, f'{cam}_rgb')
        
        os.makedirs(output_cam_rgb_path, exist_ok=True)
        
        # List all image files in the current camera folder
        for filename in os.listdir(cam_path):
            if filename.endswith('.png'):
                image_path = os.path.join(cam_path, filename)
                
                # Open the image
                img = Image.open(image_path)
                
                # Check the image size to ensure it is 1280 x 928
                if img.size != (1280, 928):
                    print(f"Skipping {image_path}: unexpected image size {img.size}")
                    continue
                
                # Split the image to get the right half
                right_half = img.crop((640, 0, 1280, 928))
                
                # Save the right half
                right_half.save(os.path.join(output_cam_rgb_path, f'{os.path.splitext(filename)[0]}{os.path.splitext(filename)[1]}'))
                
                print(f"Processed right side of {filename}")

def process_nir_images(input_folder, output_folder_ir):
    # List all camera folders
    cam_folders = [f'cam{i}' for i in range(1, 9)]

    for cam in cam_folders:
        cam_path = os.path.join(input_folder, cam)
        output_cam_ir_path = os.path.join(output_folder_ir, f'{cam}_nir')
        
        os.makedirs(output_cam_ir_path, exist_ok=True)
        
        # List all image files in the current camera folder
        for filename in os.listdir(cam_path):
            if filename.endswith('.png'):
                image_path = os.path.join(cam_path, filename)
                
                # Open the image
                img = Image.open(image_path)
                
                # Check the image size to ensure it is 1280 x 928
                if img.size != (1280, 928):
                    print(f"Skipping {image_path}: unexpected image size {img.size}")
                    continue
                
                # Split the image to get the left half
                left_half = img.crop((0, 0, 640, 928))
                
                # Save the left half
                left_half.save(os.path.join(output_cam_ir_path, f'{os.path.splitext(filename)[0]}{os.path.splitext(filename)[1]}'))
                
                print(f"Processed left side of {filename}")

# Input and output folders
input_folder = '../data'
output_folder_rgb = '../data/'
output_folder_ir = '../data/'

# Run the functions
# process_rgb_images(input_folder, output_folder_rgb)
process_nir_images(input_folder, output_folder_ir)

### Step 1: Do Radiometric Reflectance Correction on Reference Panels
### Step 2: Labelling - aka Find ROIs (Region of Interests)
### Step ???: Use AlexNet to Segment Wheat Plot Fields
### Step 3: Compute VI (Vegetation Indices)
### Step 4: Extract Data -> Summaries

## **Calculating Radiometric Reflectance on Reference Panels**

find the average value of RGB, NIR of gray panel for each image...
Then, divide each pixek in R band by average value of gray panel in red band.  repeat this for 6 bands. This becom4es reflectance image.
Register NRI images to RGB images.  You could even use the gray panel is registration points...
Calculate NDVI at each pixel.
NDFVI = (NRI-Red)/(NRI+red).  But you have 3 NIR bands.  Which one should be used?
Either segment whole wheat field from reflectance image or NDVI image.
Then, use logic to divide the wheat field into plots.
Find ststaitsics of NDVI from each plot

### Reordering CSV files in the `model_output` directory

In [11]:
import os
import pandas as pd

# Get list of CSV files in the directory
csv_files = [f for f in os.listdir('../model_output/') if f.endswith('.csv')]

# Process each CSV file
for csv_file in csv_files:
    # Read the CSV file
    file_path = f'../model_output/{csv_file}'
    df = pd.read_csv(file_path)
    
    # Remove rows with NaN in Center_X or Center_Y, and Center_X values greater than 640
    df = df.dropna(subset=['Center_X', 'Center_Y'])
    df = df[df['Center_X'] < 640]
    
    # Sort by Filename
    df = df.sort_values(by='Filename')
    
    # Save the cleaned and sorted dataframe back to the CSV file
    df.to_csv(file_path, index=False)
    
    print(f"Processed {csv_file}")
    

['cam4_centers.csv', 'cam8_centers.csv', 'cam1_centers.csv', 'cam2_centers.csv', 'cam7_centers.csv', 'cam5_centers.csv', 'cam6_centers.csv', 'cam3_centers.csv']
Processed cam4_centers.csv
Processed cam8_centers.csv
Processed cam1_centers.csv
Processed cam2_centers.csv
Processed cam7_centers.csv
Processed cam5_centers.csv
Processed cam6_centers.csv
Processed cam3_centers.csv


### Saving Plots in the `assets` directory

In [3]:
# Go through each camera and plot the Center_X and Center_Y values
output_folder = '../assets'

for i in range(1, 9):
    df = pd.read_csv(f'../model_output/cam{i}_output.csv')
    
    plt.figure(figsize=(10, 7))
    plt.scatter(df['Center_X'], df['Center_Y'], c='r', marker='o')
    plt.title(f'Camera {i} Center_X and Center_Y values')
    plt.xlabel('Center_X')
    plt.ylabel('Center_Y')
    plt.grid(True)
    plt.savefig(f'{output_folder}/cam{i}_center_x_y.png')
    plt.close()

### Performing Radiometric Reflectance Corrections

In [4]:
import pandas as pd
import cv2
import numpy as np
import os

def calculate_panel_rgb_values(csv_file, image_folder):
    # Load the CSV file
    data = pd.read_csv(csv_file)
    
    # List to store the results
    results = []

    print(f"Processing {len(data)} rows from {csv_file}")

    for index, row in data.iterrows():
        filename = row['Filename']
        center_x = row['Center_X']
        center_y = row['Center_Y']
        
        # Construct the path to the image file
        image_path = os.path.join(image_folder, filename)
        
        # Check if the file exists
        if not os.path.exists(image_path):
            # print(f"File does not exist: {image_path}")
            continue
        
        # Load the image
        image = cv2.imread(image_path)
        
        # Check if the image was loaded successfully
        if image is None:
            # print(f"Could not load image: {image_path}")
            continue
        
        # Extract a 5x5 region around the center coordinates
        x_start = max(0, center_x - 5)
        x_end = min(image.shape[1], center_x + 5)
        y_start = max(0, center_y - 5)
        y_end = min(image.shape[0], center_y + 5)
        
        region = image[y_start:y_end, x_start:x_end]
        
        # Calculate the average RGB values
        average_color = region.mean(axis=(0, 1))
        
        # Append the result
        results.append({
            'Filename': filename,
            'Center_X': center_x,
            'Center_Y': center_y,
            'Average_R': average_color[2],
            'Average_G': average_color[1],
            'Average_B': average_color[0]
        })
    
    # Convert the results to a DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df

# Define folder paths
csv_files = [f"../model_output/cam{i}_output.csv" for i in range(1, 9)]
image_folders = [f"../model_output/cam{i}" for i in range(1, 9)]

# Iterate over the CSV files and calculate the average RGB values
for csv_file, image_folder in zip(csv_files, image_folders):
    print(f"Processing {csv_file} with images from {image_folder}")
    result_df = calculate_panel_rgb_values(csv_file, image_folder)
    print(f"Processed {len(result_df)} images for {csv_file}")
    print(result_df)

Processing ../model_output/cam1_output.csv with images from ../model_output/cam1
Processing 43 rows from ../model_output/cam1_output.csv
Processed 16 images for ../model_output/cam1_output.csv
                        Filename  Center_X  Center_Y  Average_R  Average_G  \
0    date_1-6-2024_10.0.11_1.png       259        47     254.00     255.00   
1    date_1-6-2024_13.0.11_1.png       259        47     251.60     240.04   
2    date_2-6-2024_15.0.11_1.png       258        46     253.97     246.63   
3   date_24-5-2024_13.0.10_1.png       266        46     254.00     255.00   
4   date_24-5-2024_15.0.11_1.png       266        45     254.65     224.85   
5   date_25-5-2024_10.0.11_1.png       263        44     253.38     223.80   
6   date_25-5-2024_13.0.10_1.png       261        42     254.17     251.81   
7   date_26-5-2024_13.0.10_1.png       260        46     254.28     254.24   
8   date_27-5-2024_15.0.11_1.png       260        44     254.00     255.00   
9   date_28-5-2024_15.0.11_